# Introduction to the plotfile-viewer GUI

This notebook explains how to use the plotfile GUI. You can run this notebook locally by downloading it from [this link](https://github.com/plotfile/plotfile-viewer/blob/dev/docs/source/tutorials/3_Introduction-to-the-GUI.ipynb).

The GUI relies on the Jupyter notebook, and on the package `ipywidgets`. You will need to install these packages in order to be able to use the GUI.

## (optional) Preparing this notebook to run it locally

If you choose to run this notebook on your local machine, you will need to download the plotfile data files which will then be visualized. To do so, execute the following cell.

In [ ]:
import os, sys
import tarfile
from urllib.error import HTTPError, URLError
from urllib.request import urlretrieve

DATASET_BASE_URL = os.environ.get(
    "PLOTFILE_VIEWER_DATA_BASE_URL",
    "https://github.com/plotfile/plotfile-example-datasets/raw/draft",
)

def download_if_absent(dataset_name):
    """Download and decompress the requested dataset if missing."""
    if os.path.exists(dataset_name):
        return True
    tar_name = f"{dataset_name}.tar.gz"
    url = f"{DATASET_BASE_URL.rstrip('/')}/{tar_name}"
    try:
        urlretrieve(url, tar_name)
    except (HTTPError, URLError) as exc:
        print(f"Unable to download '{dataset_name}' from {url}: {exc}")
        return False
    except Exception as exc:
        print(f"Unexpected error while downloading '{dataset_name}': {exc}")
        return False
    with tarfile.open(tar_name) as tar_file:
        tar_file.extractall()
    os.remove(tar_name)
    return True

required_datasets = ['example-2d', 'example-3d', 'example-thetaMode']
missing_datasets = [name for name in required_datasets if not download_if_absent(name)]
if missing_datasets:
    message = (
        "Required tutorial datasets are unavailable: "
        + ", ".join(missing_datasets)
        + ". Place them next to this notebook or set PLOTFILE_VIEWER_DATA_BASE_URL."
    )
    if os.environ.get("PLOTFILE_VIEWER_ALLOW_EMPTY_DATA") == "1":
        print(message)
        sys.exit(0)
    raise RuntimeError(message)


In addition, we choose here to incorporate the plots inside the notebook.

In [ ]:
%matplotlib widget

## Prepare and use the GUI 

In order to start using the GUI:
- Load the class `OpenPMDTimeSeries` from the module `plotfile_viewer`

In [ ]:
from plotfile_viewer import OpenPMDTimeSeries

- Create a time series object by pointing to the folder which contains the corresponding plotfile data

In [ ]:
ts_2d = OpenPMDTimeSeries('./example-2d/hdf5/')

- Call the `slider` method

In [ ]:
ts_2d.slider()

Under the hood, this panel is calling the plotfile-viewer API with the `plot=True` option, and thus the user can quickly plot different quantities by clicking on the panel:
    - The top slider allows the user to choose the time for which to do the plot. Clicking on the `+` and `-` buttons moves from iteration to iteration.
    - The left subpanel allows to choose the plotted fields 
    - The right subpanel allows to choose the particle quantites (note that it also detects the species present in the file)
    - The `Plotting options` menu gathers some common matplotlib options.

Please try this interactive interface for yourself, in order to explore its capabilities!

## GUI for specific field geometries

Note that the above panel will adapt to the field geometry, and to the presence or absence of particles in the data files.


### 3D datasets
For instance, the code below creates a slider that explores a 3D Cartesian dataset (and contains only electrons; no ions).

In [ ]:
ts_3d = OpenPMDTimeSeries('./example-3d/hdf5/')
ts_3d.slider( fields_figure=2, particles_figure=3 )

### Cylindrical datasets with azimuthal decomposition

Similarly, the code below explores a dataset with azimuthal decomposition (2 azimuthal modes). Moreover, this dataset does not contain any particles, as reflect by the aspect of the GUI.

In [ ]:
ts_circ = OpenPMDTimeSeries('./example-thetaMode/hdf5/')
ts_circ.slider( fields_figure=4 )

Note that the interface includes buttons for the choice of modes and for the angle ``theta`` of observation.